In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from src.geometry import setup_geometry, shepp_logan
from src.plot import animate_S_D
from IPython.display import HTML


N_pixels = 256
R_obj = 125
D_sd = 1000
D_so = 500
N_det = 16
N_views = 50
view_range = 2 * np.pi
mode = "equiangular"

phantom = shepp_logan(N_pixels)

S, D, theta, beta, fan_angle, delta_beta = setup_geometry(
    D_so, D_sd, R_obj, N_det, N_views, view_range, mode
)

anim = animate_S_D(S, D, phantom)
HTML(anim.to_html5_video())

# sinogram = acquire_fanbeam_projections(phantom, S, D)

# filter = g(np.arange(-N_det / 2, N_det /2), delta_beta)

# sinogram_filt = convolve(sinogram, filter.reshape(1, -1), mode="same")

# fig, axs = plt.subplots(1,3)

# axs[0].imshow(sinogram, cmap="gray")
# axs[1].imshow(sinogram_filt, cmap="gray")
# axs[2].imshow(np.abs(sinogram - sinogram_filt), cmap="gray")

# plt.tight_layout()

In [ ]:
import numpy as np

S = S[0]
D = D[0]

# Plotting
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(S[0], S[1], "ro", label="Source")  # plot source

# Plot detector points
ax.plot(D[:, 0], D[:, 1], "bo", label="Detectors")

# Plot rays and angles
for i in range(N_det):
    ax.plot([S[0], D[i, 0]], [S[1], D[i, 1]], "k--", alpha=0.6)
    # Optional: annotate angles
    midpoint = (S + D[i]) / 2
    angle_deg = np.degrees(beta[i])
    ax.text(midpoint[0], midpoint[1], f"{angle_deg:.1f}°", fontsize=8, ha="center")

# Aesthetics
ax.set_aspect("equal")
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.legend()
ax.set_title("Fan Beam Projection Geometry")
plt.grid(True)
plt.show()

In [ ]:
plt.figure()
plt.plot(beta, label="Detector angles (beta)")
plt.title("Detector angle sweep")
plt.xlabel("Detector bin index")
plt.ylabel("Angle (rad)")
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
from src.reconstruct import compute_source_frame_coords

gamma = compute_source_frame_coords(N_pixels, D_so, theta=theta)[
    1
]  # shape: [N_pixels, N_pixels, N_views]

plt.figure()
plt.plot(beta, label="Detector beta (rad)")
plt.title("Beta - detector angle values")
plt.xlabel("Detector index")
plt.ylabel("Angle (rad)")
plt.legend()
plt.grid(True)
plt.show()

plt.figure()
plt.imshow(gamma[:, :, 0], cmap="twilight", origin="lower")
plt.title("Gamma angle map for first view")
plt.colorbar(label="Gamma (rad)")
plt.show()